In [1]:
# Add "parameters" (jupyter-notebook) tag to this cell, to allow papermill to inject different parameters
from datetime import date
it=0  #Iteration of gridsearch
# to put it all in one folder by date, will be replaced by papermill
today=date.today() 
rdate=today.strftime("%Y-%m-%d")

In [2]:
# Parameters
it = 8
rdate = "2021-08-24"


In [3]:
# See if running on Colab (for setting the correct workdir and installing all dependencies)
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  g_colab = True
else:
  print('Not running on CoLab')
  g_colab = False

Not running on CoLab


# Libraries

In [4]:
# connect to drive
if g_colab:
    from google.colab import drive
    drive.mount('/gdrive')
    %cd "/gdrive/MyDrive/1 Job/Product and Code/CogAlex 2.0/"
    
# Choose model
# Gridsearch parameters
from sklearn.model_selection import ParameterGrid

# Original results with "xlm_roberta_base"
grid = [{"model_name": ["xlm-roberta-base"],
        "datasets": [["Nothing to see here"], ["de_train_new", "de_val_new"], ["de_train_new"], ["de_val_new"],
                      ["en_train_new", "en_val_new"], ["en_train_new"], ["en_val_new"], 
                      ["de_train_new", "de_val_new","en_train_new", "en_val_new"]]},
        {"model_name": ["xlm-roberta-large", "distilbert-base-multilingual-cased", "bert-base-multilingual-uncased", "bert-base-multilingual-cased", "bert-base-cased"],
        "datasets": [["Nothing to see here"], ["de_train_new", "de_val_new","en_train_new", "en_val_new"]]}]

pg = list(ParameterGrid(grid))
print(len(pg))
model_name = pg[it]["model_name"]
datasets = pg[it]["datasets"]
print(pg[it])

18
{'datasets': ['Nothing to see here'], 'model_name': 'xlm-roberta-large'}


In [5]:
!pip install transformers
!pip install sentencepiece

You should consider upgrading via the '/home/lv71502/clang/.virtualenvs/huggingface/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/home/lv71502/clang/.virtualenvs/huggingface/bin/python -m pip install --upgrade pip' command.


In [6]:
import torch                                              #for training the model
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset
import pandas as pd                                       #for handling the data
from transformers import XLMRobertaTokenizer, AutoTokenizer              #for loading the pretrained model and tokenizer
from transformers import XLMRobertaForSequenceClassification, AutoModelForSequenceClassification
from transformers import AdamW                            
from transformers import get_linear_schedule_with_warmup
from sklearn import preprocessing                         #for label encoding
from sklearn.metrics import classification_report         #for showing performance on validation/test sets
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import ParameterGrid         #for grid search
from sklearn.model_selection import ParameterSampler      #for random search
from sklearn.utils.fixes import loguniform
import sentencepiece
import matplotlib.pyplot as plt
import time
import datetime
import random
import numpy as np
% matplotlib inline
import seaborn as sns

UsageError: Line magic function `%` not found.


# Load Data

Load data from text files 

In [ ]:
# sk learn label encoder for changing the labels to integers
labels=["ANT", "HYP", "RANDOM", "SYN"]
le = preprocessing.LabelEncoder()
le.fit(labels)

#chinese 
data_train_zh = pd.read_csv('datasets/train_chinese_data.txt', sep="\t", header=None)
data_train_zh.columns = ["Word1", "Word2", "Label"]

data_valid_zh = pd.read_csv('datasets/validgold_chinese_data.txt', sep="\t", header=None)
data_valid_zh.columns = ["Word1", "Word2", "Label"]

data_train_zh["Label"]=le.transform(data_train_zh["Label"])
data_valid_zh["Label"]=le.transform(data_valid_zh["Label"])

#english
if "en_train_new" in datasets:
    print("EN Train: NEW")
    data_train_en = pd.read_csv('datasets/train_english_data_new.txt', sep="\t", header=None, usecols=[0,1,2])
    data_train_en.columns = ["Word1", "Word2", "Label"]
else:
    print("EN Train: OLD")
    data_train_en = pd.read_csv('datasets/train_english_data.txt', sep="\t", header=None)
    data_train_en.columns = ["Word1", "Word2", "Label"]

if "en_valid_new" in datasets:
    print("EN Val: OLD")
    data_valid_en = pd.read_csv('datasets/validgold_english_data_new.txt', sep="\t", header=None, usecols=[0,1,2])
    data_valid_en.columns = ["Word1", "Word2", "Label"]

else:
    print("EN Val: OLD")
    data_valid_en = pd.read_csv('datasets/validgold_english_data.txt', sep="\t", header=None)
    data_valid_en.columns = ["Word1", "Word2", "Label"]

data_train_en["Label"]=le.transform(data_train_en["Label"])
data_valid_en["Label"]=le.transform(data_valid_en["Label"])

#german
if "de_train_new" in datasets:
    print("DE Train: NEW")
    #Cogalex 2.0 - NO DUPLICATES
    data_train_de = pd.read_csv('datasets/train_german_data_new.txt', sep="\t", header=None, usecols=[0,1,2])
    data_train_de.columns = ["Word1", "Word2", "Label"] 
else:
    print("DE Train: OLD")
    # Old data
    data_train_de = pd.read_csv('datasets/train_german_data.txt', sep="\t", header=None)
    data_train_de.columns = ["Word1", "Word2", "Label"]

if "de_val_new" in datasets:
    print("DE Val: NEW")
    #Cogalex 2.0 - NO DUPLICATES
    data_valid_de = pd.read_csv('datasets/validgold_german_data_new.txt', sep="\t", header=None, usecols=[0,1,2])
    data_valid_de.columns = ["Word1", "Word2", "Label"]
else:
    print("DE Val: OLD")
    # Old data
    data_valid_de = pd.read_csv('datasets/validgold_german_data.txt', sep="\t", header=None)
    data_valid_de.columns = ["Word1", "Word2", "Label"]


data_train_de["Label"]=le.transform(data_train_de["Label"])
data_valid_de["Label"]=le.transform(data_valid_de["Label"])


# all together
data_train_all=pd.concat([data_train_zh, data_train_en, data_train_de])
data_train_all=data_train_all.reset_index(drop=True)
data_valid_all=pd.concat([data_valid_zh, data_valid_en, data_valid_de])
data_valid_all=data_valid_all.reset_index(drop=True)

In [ ]:
data_train_en

In [ ]:
# print class distribution 
data_train_all["Label"].value_counts().plot(kind='bar', title='Count (target)')

# Tokenize

Tokenize Data

- encode in the right format for XLM-Roberta ( <s\> = sentence beginning, </s\> end of sentence/sentence seperator)

- Truncate/Padding so everything has the same length

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
if "gpt2" in model_name:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token_id

In [ ]:
max_len=64

def tokenizer_xlm(data, max_len):
  labels_ = []
  input_ids_ = []
  attn_masks_ = []

  # for each datasample:
  for index, row in data.iterrows():

      word1 = row['Word1']
      word2 = row['Word2']

      # create requiered input, i.e. ids and attention masks
      encoded_dict = tokenizer.encode_plus(word1, word2, 
                                                max_length=max_len, 
                                                padding='max_length',
                                                truncation=True, 
                                                return_tensors='pt')

      # add encoded sample to lists
      input_ids_.append(encoded_dict['input_ids'])
      attn_masks_.append(encoded_dict['attention_mask'])
      labels_.append(row['Label'])
      
  # Convert each Python list of Tensors into a 2D Tensor matrix.
  input_ids_ = torch.cat(input_ids_, dim=0)
  attn_masks_ = torch.cat(attn_masks_, dim=0)

  # labels to tensor
  labels_ = torch.tensor(labels_)

  print('Encoder finished. {:,} examples.'.format(len(labels_)))
  return input_ids_, attn_masks_, labels_

In [ ]:
# a small test to see how the tokenizer works
w1 = "Tiger"
w2 = "Animal"

# Encode the two sentences together.
encoded = tokenizer.encode_plus(w1, w2)

# Print the IDs of the resulting tokens.
print ("Input IDs:      ", encoded['input_ids'])

# Convert the token IDs back to strings so we can check them out.
print ("Tokens:         ", tokenizer.convert_ids_to_tokens(encoded['input_ids']))

# The tokenizer returns an attention mask, which masks out PAD tokens. 
# Since we aren't doing any padding yet, the mask is just all 1s. 
print ("\nAttention Mask: ", encoded['attention_mask'])


In [ ]:
data_train_all.isnull().values.any()

In [ ]:
data_valid_all.isnull().values.any()

In [ ]:
# tokenize data

#all
print("All")
input_ids_train_all, attn_masks_train_all, labels_train_all = tokenizer_xlm(data_train_all, max_len)
input_ids_valid_all, attn_masks_valid_all, labels_valid_all = tokenizer_xlm(data_valid_all, max_len)

#zh
print("zh")
input_ids_train_zh, attn_masks_train_zh, labels_train_zh = tokenizer_xlm(data_train_zh, max_len)
input_ids_valid_zh, attn_masks_valid_zh, labels_valid_zh = tokenizer_xlm(data_valid_zh, max_len)

#en
print("en")
input_ids_train_en, attn_masks_train_en, labels_train_en = tokenizer_xlm(data_train_en, max_len)
input_ids_valid_en, attn_masks_valid_en, labels_valid_en = tokenizer_xlm(data_valid_en, max_len)

#de
print("de")
input_ids_train_de, attn_masks_train_de, labels_train_de = tokenizer_xlm(data_train_de, max_len)
input_ids_valid_de, attn_masks_valid_de, labels_valid_de = tokenizer_xlm(data_valid_de, max_len)

In [ ]:
# Combine the training inputs into a TensorDataset.

#all
tensor_data_train_all = TensorDataset(input_ids_train_all, attn_masks_train_all, labels_train_all)
tensor_data_valid_all = TensorDataset(input_ids_valid_all, attn_masks_valid_all, labels_valid_all)
#zh
tensor_data_train_zh = TensorDataset(input_ids_train_zh, attn_masks_train_zh, labels_train_zh)
tensor_data_valid_zh = TensorDataset(input_ids_valid_zh, attn_masks_valid_zh, labels_valid_zh)
#en
tensor_data_train_en = TensorDataset(input_ids_train_en, attn_masks_train_en, labels_train_en)
tensor_data_valid_en = TensorDataset(input_ids_valid_en, attn_masks_valid_en, labels_valid_en)
#de
tensor_data_train_de = TensorDataset(input_ids_train_de, attn_masks_train_de, labels_train_de)
tensor_data_valid_de = TensorDataset(input_ids_valid_de, attn_masks_valid_de, labels_valid_de)

In [ ]:
#prepare pytorch dataloaders

batch_size = 32

#all
train_dataloader_all = DataLoader(tensor_data_train_all, sampler = RandomSampler(tensor_data_train_all), batch_size = batch_size) #random sampling
validation_dataloader_all = DataLoader(tensor_data_valid_all, sampler = SequentialSampler(tensor_data_valid_all),batch_size = batch_size ) #sequential sampling
#zh
train_dataloader_zh = DataLoader(tensor_data_train_zh, sampler = RandomSampler(tensor_data_train_zh), batch_size = batch_size)
validation_dataloader_zh = DataLoader(tensor_data_valid_zh, sampler = SequentialSampler(tensor_data_valid_zh),batch_size = batch_size)
#en
train_dataloader_en = DataLoader(tensor_data_train_en, sampler = RandomSampler(tensor_data_train_en), batch_size = batch_size)
validation_dataloader_en = DataLoader(tensor_data_valid_en, sampler = SequentialSampler(tensor_data_valid_en),batch_size = batch_size)
#de
train_dataloader_de = DataLoader(tensor_data_train_de, sampler = RandomSampler(tensor_data_train_de), batch_size = batch_size)
validation_dataloader_de = DataLoader(tensor_data_valid_de, sampler = SequentialSampler(tensor_data_valid_de),batch_size = batch_size)

# Training the Classifier (Finetunig the Model)

In [ ]:
#load the pretrained model provided by HuggingFace with an added untrained classification head for 4 classes
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)
#model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=4)

In [ ]:
#choose training set used for this single training example (original training data for final model)
train_dataloader = train_dataloader_all
print("Training Samples:",len(train_dataloader.dataset))

In [ ]:
# connect to GPU 
device = torch.device('cuda')
# copy weights onto gpu
desc = model.to(device)

print('Connected to GPU:', torch.cuda.get_device_name(0))

Set parameters (learning rate & epochs)

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,   #do work well: 2e-5 with 5-7 epochs for trainall, 1e-5
                  eps = 1e-8   # 1e-8.
                  # weight_decay = 0          
                )

In [ ]:
# number of training epochs
epochs = 15

# number of batches x epochs
total_steps = len(train_dataloader) * epochs
print("total steps:", total_steps)

#scheduler for lr
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,   #start low and increase learning rate during these steps
                                            num_training_steps = total_steps)

Formatting functions

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))  


Validation function

In [ ]:
def validate(validation_dataloader, model, verbose): 
  
  # put model in evaluation mode 
  model.eval()

  # Tracking variables 
  total_eval_loss = 0


  predictions, true_labels = [], []

  # Evaluate data for one epoch
  for batch in validation_dataloader:
          
      # Unpack training batch and copy the tensors to the gpu
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)
          
      # no backprop needed
      with torch.no_grad():        

          # forward pass
          if "distilbert" in model.name_or_path:
              output = model(b_input_ids, 
                              attention_mask=b_input_mask, 
                              labels=b_labels)
          else:
              output = model(b_input_ids, 
                              token_type_ids=None, 
                              attention_mask=b_input_mask, 
                              labels=b_labels)
          
          loss=output.loss
          logits=output.logits
              
      # add up loss
      total_eval_loss += loss.item()

      # on cpu
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()

      # save preds/true labels
      predictions.append(logits)
      true_labels.append(label_ids)

  # results of the whole validation set
  flat_predictions = np.concatenate(predictions, axis=0)
  flat_true_labels = np.concatenate(true_labels, axis=0)

  # logit to label
  predicted_labels = np.argmax(flat_predictions, axis=1).flatten()

  # print classification report
  if verbose:
    print(classification_report(flat_true_labels, predicted_labels, target_names=labels))

  # Calculate the validation accuracy, macro f1, and weighted f1 without RANDOM
  val_accuracy = (predicted_labels == flat_true_labels).mean()
  macroF1 = f1_score(flat_true_labels, predicted_labels, average='macro')
  weightedF1=f1_score(flat_true_labels, predicted_labels, average='weighted')
  print("\t Weighted F1 (no random):", weightedF1)

  # Calculate the average loss over all of the batches.
  avg_val_loss = total_eval_loss / len(validation_dataloader)

  # plot confusion matrix
  if verbose:
    print(confusion_matrix(flat_true_labels, predicted_labels, labels=[0,1,2,3]))

  return avg_val_loss, val_accuracy, macroF1, weightedF1
      

Training

In [ ]:
def train_model(epochs, model, train_dataloader, validation_dataloader_set, random_seed, verbose, optimizer, scheduler):

  seed_val = random_seed

  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

  # mostly contains scores about how the training went for each epoch
  training_stats = []

  # total training time
  total_t0 = time.time()

  print('\033[1m'+"================ Model Training ================"+'\033[0m')

  # For each epoch...
  for epoch_i in range(0, epochs):

      print("")
      print('\033[1m'+'======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs)+'\033[0m')

      t0 = time.time()

      # summed training loss of the epoch
      total_train_loss = 0


      # model is being put into training mode as mechanisms like dropout work differently during train and test time
      model.train()

      # iterrate over batches
      for step, batch in enumerate(train_dataloader):

          # unpack training batch at load it to gpu (device)  
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)

          # clear gradients before calculating new ones
          model.zero_grad()        

          # forward pass with current batch
          if "distilbert" in model.name_or_path:
              output = model(b_input_ids, 
                              attention_mask=b_input_mask, 
                              labels=b_labels)
          else:
              output = model(b_input_ids, 
                              token_type_ids=None, 
                              attention_mask=b_input_mask, 
                              labels=b_labels)
          
          loss=output.loss
          logits=output.logits

          # add up the loss
          total_train_loss += loss.item()

          # calculate new gradients
          loss.backward()

          # gradient clipping (not bigger than)
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          # Update the networks weights based on the gradient as well as the optimiziers parameters
          optimizer.step()

          # lr update
          scheduler.step()

      # avg loss over all batches
      avg_train_loss = total_train_loss / len(train_dataloader)            
      
      # training time of this epoch
      training_time = format_time(time.time() - t0)

      print("")
      print("  Average training loss: {0:.2f}".format(avg_train_loss))
      print("  Training epoch took: {:}".format(training_time))
          
  
      # VALIDATION

      #all
      print("evaluate on all")
      avg_val_loss_all, val_accuracy_all, macroF1_all, weightedF1_no_random_all = validate(validation_dataloader_set[0], model, verbose)   
      #zh
      print("evaluate on zh")
      avg_val_loss_zh, val_accuracy_zh, macroF1_zh, weightedF1_no_random_zh = validate(validation_dataloader_set[1], model, verbose) 
      #en
      print("evaluate on en")
      avg_val_loss_en, val_accuracy_en, macroF1_en, weightedF1_no_random_en = validate(validation_dataloader_set[2], model, verbose) 
      #de
      print("evaluate on de")
      avg_val_loss_de, val_accuracy_de, macroF1_de, weightedF1_no_random_de = validate(validation_dataloader_set[3], model, verbose)  
       

      print('\033[1m'+ "  Validation Loss All: {0:.2f}".format(avg_val_loss_all) + '\033[0m')

      training_stats.append(
          {
              'epoch': epoch_i + 1,
              'Training Loss': avg_train_loss,
              'Valid. Loss all': avg_val_loss_all,
              'Valid. Accur. all': val_accuracy_all,
              'Weigh_F1 all (no random)': weightedF1_no_random_all, 
              'Macro F1 all': macroF1_all,
              'Weigh_F1 en': weightedF1_no_random_en,
              'Weigh_F1 de': weightedF1_no_random_de,
              'Weigh_F1 zh': weightedF1_no_random_zh,
              'Training Time': training_time,
          }
      )

  print("\n\nTraining complete!")
  print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
  
  return training_stats


In [ ]:
validation_dataloader_set=[validation_dataloader_all, validation_dataloader_zh, validation_dataloader_en, validation_dataloader_de]

In [ ]:
# start training
training_stats=train_model(epochs=epochs, 
                           model=model, 
                           train_dataloader=train_dataloader, 
                           validation_dataloader_set=validation_dataloader_set,
                           random_seed=42,
                           verbose=True,
                           scheduler=scheduler,
                           optimizer=optimizer)  

# Evaluation Statistics

In [ ]:
# Table with training stats
pd.set_option('precision', 3)
df_stats = pd.DataFrame(data=training_stats)
df_stats = df_stats.set_index('epoch')
df_stats

In [ ]:
# Plot Trainingloss, Validation loss and Weighted F1 score for each epoch

sns.set(style='darkgrid')
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss all'], 'g-o', label="Validation")
plt.plot(df_stats['Weigh_F1 all (no random)'], 'r-o', label="Weighted F1 (all, no rand)")
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# save model
import os
if g_colab:
    workdir = os.getcwd()
else:
    workdir = os.getcwd().replace("/home/","/binfl/")

print(workdir)

In [ ]:
os.makedirs(f"{workdir}/saved_models/{model_name}_{it}", exist_ok=True)
os.makedirs("./stats/", exist_ok=True)
PATH = f"{workdir}/saved_models/{model_name}_{it}"
df_stats.to_csv(f"./stats/{model_name}_{it}")
model.save_pretrained(PATH)